In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper.py


In [4]:
%%writefile test.dat

1	hello world
2	hello again
42	Привет мир!
3	Another example with heLLo
4	For The Horde!
5	Hello again

Overwriting test.dat


In [5]:
cat test.dat | python2 ./mapper.py | sort

reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
again	1
again	1
another	1
example	1
for	1
hello	1
hello	1
hello	1
hello	1
horde!	1
the	1
with	1
world	1
мир!	1
привет	1


In [6]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Writing reducer.py


In [7]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [8]:
cat test.dat | python2 ./mapper.py | sort | python2 ./reducer.py

reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
reporter:counter:Wiki stats,Total words,1
again	2
another	1
example	1
for	1
hello	4
horde!	1
the	1
with	1
world	1
мир!	1
привет	1


In [9]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2018-08-01 17:15 /data/wiki/en_articles_part


In [10]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

0%however	1
0&\mathrm{if	1
0(8)320-1234	1
0)).(1	2
0,03	1
0,1,...,n	1
0,1,0	1
0,1,\dots,n	1
0,5	1
0,50	1


rm: `wordcount_result_1555214183812849': No such file or directory
19/04/14 03:56:27 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/04/14 03:56:27 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/04/14 03:56:28 INFO mapred.FileInputFormat: Total input files to process : 1
19/04/14 03:56:29 INFO mapreduce.JobSubmitter: number of splits:2
19/04/14 03:56:29 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1555208747664_0001
19/04/14 03:56:30 INFO impl.YarnClientImpl: Submitted application application_1555208747664_0001
19/04/14 03:56:30 INFO mapreduce.Job: The url to track the job: http://b638d3c70fae:8088/proxy/application_1555208747664_0001/
19/04/14 03:56:30 INFO mapreduce.Job: Running job: job_1555208747664_0001
19/04/14 03:56:37 INFO mapreduce.Job: Job job_1555208747664_0001 running in uber mode : false
19/04/14 03:56:37 INFO mapreduce.Job:  map 0% reduce 0%
19/04/14 03:56:53 INFO mapreduce.Job:  map 32% reduce 0%
19/04/14 03:5